引入套件

In [1]:
from gensim.models import KeyedVectors
import pandas as pd


引入pre-train model(https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g)(記得下載)
、dataset

In [2]:
model = KeyedVectors.load_word2vec_format('model/GoogleNews-vectors-negative300.bin.gz', binary=True)

Word353 = pd.read_csv("data/WordSimilarity/combined.csv")

以word2vec對每行的兩個word預測相似度

In [3]:
def extract_words(row):
    return  (model.similarity(row['Word 1'], row['Word 2'])+1)*5 
   
df = Word353.apply(extract_words, axis=1)
merge_df = pd.concat([df, Word353], axis=1)
merge_df = merge_df.rename(columns={0: 'Word2vec_score'})

merge_df

,Word2vec_score,Word 1,Word 2,Human (mean)
0,6.319689,love,sex,6.77
1,7.586481,tiger,cat,7.35
2,10.000000,tiger,tiger,10.00
3,6.817313,book,paper,7.46
4,6.981958,computer,keyboard,7.62
...,...,...,...,...
348,5.647397,shower,flood,6.03
349,6.813604,weather,forecast,8.34
350,5.726142,disaster,area,6.25
351,6.483178,governor,office,6.34


計算與人類 Human (mean)的差異

In [4]:
def calculate_difference(row):
    return abs(row['Word2vec_score'] - row['Human (mean)'])

merge_df['diff_abs'] = merge_df.apply(calculate_difference, axis=1)

print(merge_df)
diff_abs_mean = merge_df['diff_abs'].mean()
print("diff_abs 平均值：", diff_abs_mean)

     Word2vec_score        Word 1    Word 2  Human (mean)      diff_abs
0          6.319689          love       sex          6.77  4.503114e-01
1          7.586481         tiger       cat          7.35  2.364810e-01
2         10.000000         tiger     tiger         10.00  2.980232e-07
3          6.817313          book     paper          7.46  6.426870e-01
4          6.981958      computer  keyboard          7.62  6.380418e-01
..              ...           ...       ...           ...           ...
348        5.647397        shower     flood          6.03  3.826033e-01
349        6.813604       weather  forecast          8.34  1.526396e+00
350        5.726142      disaster      area          6.25  5.238584e-01
351        6.483178      governor    office          6.34  1.431778e-01
352        5.715467  architecture   century          3.78  1.935467e+00

[353 rows x 5 columns]
diff_abs 平均值： 1.3775137043640377


做成csv

In [9]:

merge_df.to_csv("result/word2vec_353.csv")